In [1]:
import tensorflow as tf
import numpy as np


C:\Users\zhaon\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [94]:
#get cifar10
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data();
print(x_train.shape)
print(y_train.shape)


(50000, 32, 32, 3)
(50000, 1)


In [44]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import scale
onehot_encoder = OneHotEncoder(sparse=False)
y_train_ohe = onehot_encoder.fit_transform(y_train);
print(y_train_ohe.shape)
print(type(y_train_ohe))
y_test_ohe =onehot_encoder.fit_transform(y_test); 
x_train = scale(x_train, axis = 0);
x_test = scale(x_test, axis = 0);

(50000, 10)
<class 'numpy.ndarray'>


In [ ]:
#take a batch
batch_size = 1000
[N,H,W,C] = x_train.shape;
num_batches = int(N/batch_size);


# define a resnet architecture using layers api
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape = [None, 32, 32, 3], name = 'X')
input_labels = tf.placeholder(tf.int32, shape = [None, 10], name = 'y')
inputs = X;
for i in range(30):
    filters = 3; kernel_size = (3,3);
    z1 = tf.layers.conv2d(inputs, filters, kernel_size, padding = 'same');
    #z2 = tf.layers.max_pooling2d(z1, (2,2), strides = (1,1))
    z3 = tf.layers.batch_normalization(z1); 
    #input goes before relu
    if(i%2 == 0):
        z4 = tf.nn.relu(z3+inputs);
    else:
        z4 = tf.nn.relu(z3);

    ## to make this a resnet, just add in the inputs here (f(x)+x); 
    # the challenge is that the input has been conved and maxpooled..
    #doesn't seem like resnetting the conv layers works
    inputs = z4;

        

flat = tf.layers.flatten(z4);

d1 = tf.layers.dense(flat, 100, activation = tf.nn.relu);

scores = tf.layers.dense(d1, 10);

loss= tf.nn.softmax_cross_entropy_with_logits_v2(labels = input_labels, logits = scores);
loss = tf.reduce_mean(loss);
optimizer = tf.train.AdamOptimizer();
optimizer = optimizer.minimize(loss);

init = tf.global_variables_initializer()
sess = tf.Session();
sess.run(init)
epochs = 20; print(num_batches)
for i in range(epochs):
    for j in range(num_batches):
        X_batch = x_train[j*batch_size:(j+1)*batch_size, :, :, :];
        y_batch = y_train_ohe[j*batch_size:(j+1)*batch_size, :];
        y_batch_flat = y_train[j*batch_size:(j+1)*batch_size, :]        
        sess.run(optimizer, feed_dict = {X: X_batch, input_labels: y_batch})
        #if(j == num_batches-1): print(sess.run(loss, feed_dict = {X: X_batch, input_labels: y_batch}))


    print(sess.run(loss, feed_dict = {X: X_batch, input_labels: y_batch}))
        


50
1.8524355
1.8524355
1.7374415
1.7374415
1.6425681
1.6425681
1.5757244
1.5757244
1.5284251
1.5284251
1.4797028
1.4797028
1.4434774
1.4434774
1.4025108
1.4025108


In [96]:
#test loss
print(sess.run(loss, feed_dict = {X: x_test, input_labels: y_test_ohe}))

print(y_batch.shape)
#get test accuracy;
test_scores = sess.run(scores, feed_dict = {X: x_test, input_labels: y_test_ohe});
test_labels = np.argmax(test_scores, axis = 1);
print(0.001*np.sum(test_labels == np.reshape(y_test_flat, (10000, ))))

print(y_batch_flat.shape)
print(test_labels.shape)

4.8761163
(1000, 10)
1.0
(1000, 1)
(1000,)
